In [76]:
!pip install googletrans

In [77]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
print("done")


done


In [78]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
#aiml_key = os.getenv('AIML_API_KEY')

# Check the key

if not api_key:
    print("No API key was found ")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key ")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - ")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [79]:
openai = OpenAI()


In [80]:
# A class to represent a Webpage


# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [81]:
# Let's try one out.
link=input("paste the website link here")
site = Website(link)
print(site.title)
print(site.text)

paste the website link herehttps://huggingface.co/
Hugging Face – The AI community building the future.
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen-Image-Edit
Updated
3 days ago
•
16.8k
•
905
deepseek-ai/DeepSeek-V3.1-Base
Updated
about 4 hours ago
•
5.68k
•
757
google/gemma-3-270m
Updated
7 days ago
•
49.1k
•
538
tencent/Hunyuan-GameCraft-1.0
Updated
2 days ago
•
74
•
437
google/gemma-3-270m-it
Updated
7 days ago
•
67.9k
•
310
Browse 1M+ models
Spaces
Running
12.1k
12.1k
DeepSite v2
🐳
Generate any application with DeepSeek
Running
on
Zero
200
200
Qwen Image Edit
✒
Edit images based on user instructions
Running
on
Zero
MCP
260
260
Wan2.2 14B Fast
🎥
generate a video from an image with a text prompt
Running
on
Zero
691
691
Qwen Image
🖼

## Types of prompts



Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [82]:
# Define our system prompt -

system_prompt = '''
You are an assistant that analyzes the content of a website and provides a concise, user-friendly summary.

Guidelines:

Focus only on the main content of the webpage.

Ignore navigation elements, menus, headers, footers, and repeated boilerplate text.

Present the summary in Markdown format for readability.

Keep the summary short and clear, highlighting the key information or purpose of the page.

If the page has multiple main sections, provide a brief bullet-point breakdown.

Do not include promotional or irrelevant filler text unless it contributes to the main content.'''
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a  summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt
print(user_prompt_for(site))

You are looking at a website titled Hugging Face – The AI community building the future.
The contents of this website is as follows; please provide a  summary of this website in markdown. If it includes news or announcements, then summarize these too.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen-Image-Edit
Updated
3 days ago
•
16.8k
•
905
deepseek-ai/DeepSeek-V3.1-Base
Updated
about 4 hours ago
•
5.68k
•
757
google/gemma-3-270m
Updated
7 days ago
•
49.1k
•
538
tencent/Hunyuan-GameCraft-1.0
Updated
2 days ago
•
74
•
437
google/gemma-3-270m-it
Updated
7 days ago
•
67.9k
•
310
Browse 1M+ models
Spaces
Running
12.1k
12.1k
DeepSite v2
🐳
Generate any application with DeepSeek
Running
on
Zero
200
200
Qwen Image Edit
✒
Edit images based on u

## And now let's build useful messages for GPT-4o-mini, using a function

In [83]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]
messages_for(site)

[{'role': 'system',
  'content': '\nYou are an assistant that analyzes the content of a website and provides a concise, user-friendly summary.\n\nGuidelines:\n\nFocus only on the main content of the webpage.\n\nIgnore navigation elements, menus, headers, footers, and repeated boilerplate text.\n\nPresent the summary in Markdown format for readability.\n\nKeep the summary short and clear, highlighting the key information or purpose of the page.\n\nIf the page has multiple main sections, provide a brief bullet-point breakdown.\n\nDo not include promotional or irrelevant filler text unless it contributes to the main content.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Hugging Face – The AI community building the future.\nThe contents of this website is as follows; please provide a  summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nL

## Time to bring it together - the API for OpenAI is very simple!

In [84]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content



In [85]:
url=input("paste the website link here")
summary = summarize(url)
display(Markdown(summary))



paste the website link herehttps://huggingface.co/


# Hugging Face Summary

Hugging Face is a collaborative platform designed for the machine learning community to share, develop, and deploy models, datasets, and applications. It fosters collaboration and provides tools and resources for AI enthusiasts and professionals.

## Key Features

- **Models and Datasets**: Access over 1 million models and 250,000 datasets for various machine learning tasks.
- **Spaces**: Run multiple applications like image editing, text generation, and video creation.
- **Open Source Collaboration**: Contribute to foundational tools in machine learning, such as Transformers and Diffusers.
- **Enterprise Solutions**: Offers paid computing resources, team collaboration features, and enterprise-grade support.

## Recent Updates

- **Trending Models**: 
  - **Qwen/Qwen-Image-Edit**: Updated 3 days ago.
  - **DeepSeek-V3.1-Base**: Updated about 4 hours ago.
  - **Google/gemma-3-270m**: Updated 7 days ago.
  
- **Spaces Running**: Include various applications like Qwen Image for generating images from text prompts and DeepSite for application generation.

## Community Engagement

- **Usage**: Over 50,000 organizations utilize Hugging Face, including major enterprises like Google and Microsoft.
- **Collaboration**: Users can host public models and datasets, facilitating community participation and feedback.

Hugging Face serves as a comprehensive hub for machine learning development, emphasizing community collaboration and resource-sharing.

In [86]:


# Your original markdown summary in English
markdown_text = summary
system_prompt='''
Translate the following text from English to Hindi.
Keep all Markdown formatting the same.
Only translate the text, do not change symbols or structure.

'''
# Ask GPT-4o-mini to translate while keeping Markdown structure
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": markdown_text}
    ]
)


translated_markdown = response.choices[0].message.content
print(translated_markdown)



# Hugging Face सारांश

Hugging Face एक सहयोगी मंच है जिसे मशीन लर्निंग समुदाय के लिए मॉडल, डेटासेट और अनुप्रयोगों को साझा, विकसित और तैनात करने के लिए डिज़ाइन किया गया है। यह सहयोग को बढ़ावा देता है और एआई उत्साही लोगों और पेशेवरों के लिए उपकरण और संसाधन प्रदान करता है।

## मुख्य विशेषताएँ

- **मॉडल और डेटासेट**: विभिन्न मशीन लर्निंग कार्यों के लिए 1 मिलियन से अधिक मॉडल और 250,000 डेटासेट तक पहुँचें।
- **स्पेस**: छवि संपादन, पाठ पीढ़ी और वीडियो निर्माण जैसी कई अनुप्रयोगों को चलाएँ।
- **ओपन सोर्स सहयोग**: मशीन लर्निंग में बुनियादी उपकरणों जैसे कि Transformers और Diffusers में योगदान करें।
- **उद्यम समाधान**: भुगतान किए गए कंप्यूटिंग संसाधन, टीम सहयोग सुविधाएँ, और उद्यम-ग्रेड समर्थन प्रदान करता है।

## हाल के अपडेट

- **प्रवृत्त मॉडल**: 
  - **Qwen/Qwen-Image-Edit**: 3 दिन पहले अपडेट किया गया।
  - **DeepSeek-V3.1-Base**: लगभग 4 घंटे पहले अपडेट किया गया।
  - **Google/gemma-3-270m**: 7 दिन पहले अपडेट किया गया।
  
- **चालू स्पेस**: Qwen Image जैसी विभिन्न अनुप्रयोगों को शामिल करता है जो पाठ

In [87]:
display(Markdown(translated_markdown))

# Hugging Face सारांश

Hugging Face एक सहयोगी मंच है जिसे मशीन लर्निंग समुदाय के लिए मॉडल, डेटासेट और अनुप्रयोगों को साझा, विकसित और तैनात करने के लिए डिज़ाइन किया गया है। यह सहयोग को बढ़ावा देता है और एआई उत्साही लोगों और पेशेवरों के लिए उपकरण और संसाधन प्रदान करता है।

## मुख्य विशेषताएँ

- **मॉडल और डेटासेट**: विभिन्न मशीन लर्निंग कार्यों के लिए 1 मिलियन से अधिक मॉडल और 250,000 डेटासेट तक पहुँचें।
- **स्पेस**: छवि संपादन, पाठ पीढ़ी और वीडियो निर्माण जैसी कई अनुप्रयोगों को चलाएँ।
- **ओपन सोर्स सहयोग**: मशीन लर्निंग में बुनियादी उपकरणों जैसे कि Transformers और Diffusers में योगदान करें।
- **उद्यम समाधान**: भुगतान किए गए कंप्यूटिंग संसाधन, टीम सहयोग सुविधाएँ, और उद्यम-ग्रेड समर्थन प्रदान करता है।

## हाल के अपडेट

- **प्रवृत्त मॉडल**: 
  - **Qwen/Qwen-Image-Edit**: 3 दिन पहले अपडेट किया गया।
  - **DeepSeek-V3.1-Base**: लगभग 4 घंटे पहले अपडेट किया गया।
  - **Google/gemma-3-270m**: 7 दिन पहले अपडेट किया गया।
  
- **चालू स्पेस**: Qwen Image जैसी विभिन्न अनुप्रयोगों को शामिल करता है जो पाठ संकेतों से चित्र उत्पन्न करता है और DeepSite जो अनुप्रयोग निर्माण के लिए है।

## समुदाय सहभागिता

- **उपयोग**: 50,000 से अधिक संगठन Hugging Face का उपयोग कर रहे हैं, जिसमें Google और Microsoft जैसे प्रमुख उद्यम शामिल हैं।
- **सहयोग**: उपयोगकर्ता सार्वजनिक मॉडल और डेटासेट की मेज़बानी कर सकते हैं, जिससे समुदाय की भागीदारी और फीडबैक को बढ़ावा मिलता है।

Hugging Face मशीन लर्निंग विकास के लिए एक व्यापक केंद्र के रूप में सेवा करता है, जो समुदाय सहयोग और संसाधन साझा करने पर जोर देता है।